### Imports

In [ ]:
import warnings

import networkx as nx
from qiskit import QuantumCircuit, qpy
from qiskit.primitives import Sampler
from qiskit_algorithms.minimum_eigensolvers import QAOA
from qiskit_algorithms.optimizers import SLSQP
from qiskit_optimization.applications import Maxcut

warnings.filterwarnings("ignore", category=DeprecationWarning)
from pathlib import Path

import numpy as np
from qiskit import transpile
from qiskit.circuit.library import n_local
from qiskit_aer import AerSimulator

### Helper functions

In [ ]:
current_dir = Path.cwd()


def save_circuit_as_qpy(circuit, save_dir, circuit_name) -> None:
    """Save a quantum circuit as a QPY file (Qiskit's native format)."""

    # Associate the circuit with its name
    circuit.name = circuit_name

    # Ensure directory exists
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    file_path = save_dir / f"{circuit_name}.qpy"

    # Check if the file already exists
    if file_path.exists():
        print(f"File {file_path} already exists.")
        return

    # Save file
    with Path.open(file_path, "wb") as f:
        qpy.dump(circuit, f)

    # Load file
    with Path.open(file_path, "rb") as f:
        qpy.load(f)

    print(f"Circuit saved to {file_path}")

### Circuit generator functions

In [ ]:
def generate_qnn_circuits(max_qubits=5, max_depth=10, seed=42, single_qubit_gates=None, multi_qubit_gates=None) -> None:
    """Generate QNN circuits with a given number of qubits and depth."""

    if multi_qubit_gates is None:
        multi_qubit_gates = ["cx"]
    if single_qubit_gates is None:
        single_qubit_gates = ["sx", "ry"]
    reached_max_depth, n_layers = False, 0
    for n_qubits in range(2, max_qubits + 1):
        for entanglement in ["full", "linear", "sca", "reverse_linear", "circular", "pairwise"]:
            while not reached_max_depth:
                n_layers += 1

                circ_name = f"twolocal_{n_qubits}qubits_{n_layers}layers_{entanglement}_entanglement"

                # Create circuit
                circ = n_local(
                    num_qubits=n_qubits,
                    rotation_blocks=single_qubit_gates,
                    entanglement_blocks=multi_qubit_gates,
                    entanglement=entanglement,
                    reps=n_layers,
                    parameter_prefix="θ",
                    insert_barriers=False,
                    overwrite_block_parameters=True,
                    skip_final_rotation_layer=False,
                    skip_unentangled_qubits=False,
                    name=circ_name,
                )
                # Add final measurements
                circ.measure_all()

                if circ.depth() > max_depth:
                    reached_max_depth = True
                    break

                # Set random parameters to the rotation gates
                rng = np.random.default_rng(seed)
                param_values = rng.uniform(0, 2 * np.pi, size=len(circ.parameters))
                param_dict = dict(zip(circ.parameters, param_values, strict=False))
                circ = circ.assign_parameters(param_dict)

                # Save the circuit
                save_circuit_as_qpy(circ, current_dir / "qnn_circuits", circ_name)

            # Reset the flag and layers counter
            reached_max_depth, n_layers = False, 0

In [ ]:
def generate_qaoa_circuits(
    max_qubits=5, max_depth=50, seed=42, single_qubit_gates=None, multi_qubit_gates=None
) -> None:
    """Generate QAOA circuits for MaxCut problems."""

    if multi_qubit_gates is None:
        multi_qubit_gates = ["cx"]
    if single_qubit_gates is None:
        single_qubit_gates = ["sx", "ry"]
    reached_max_depth, n_layers = False, 0
    for n_qubits in range(2, max_qubits + 1):  # Number of nodes aka qubits
        for degree in range(2, n_qubits):  # This changes the problem Hamiltonian (and its solution)
            if (n_qubits * degree) % 2:
                continue  # must be even

            while reached_max_depth is False:
                n_layers += 1

                circ_name = f"qaoa_{n_qubits}qubits_{n_layers}layers_{degree}degree"

                # Generate a random regular graph for MaxCut
                graph = nx.random_regular_graph(d=degree, n=n_qubits, seed=seed)
                maxcut = Maxcut(graph)

                mixer_circle = QuantumCircuit(n_qubits)
                mixer_circle.sx(range(n_qubits))

                # Create QAOA solver
                qaoa = QAOA(sampler=Sampler(), reps=n_layers, optimizer=SLSQP(maxiter=25))

                # Compute minimum eigenvalue for the MaxCut Ising model
                ising_op, _offset = maxcut.to_quadratic_program().to_ising()
                qaoa_result = qaoa.compute_minimum_eigenvalue(operator=ising_op)

                # Create the QAOA ansatz circuit and assign optimal parameters -> get actual results when running the circuit
                qaoa_circuit = qaoa.ansatz.assign_parameters(qaoa_result.optimal_point)

                transpiled_circuit = transpile(
                    qaoa_circuit, optimization_level=3, basis_gates=single_qubit_gates + multi_qubit_gates
                )

                # Skip circuits exceeding the maximum depth
                if transpiled_circuit.depth() > max_depth:
                    reached_max_depth = True
                    break

                # Save circuit as QPY file
                save_circuit_as_qpy(transpiled_circuit, current_dir / "qaoa_circuits", circ_name)

            # Reset the flag and layers counter
            reached_max_depth, n_layers = False, 0

### Setup

In [ ]:
# For reproducibility
seed = 111

# QPU (or simulation) limits
max_qubits = 5
max_depth = 20

# Native gates (currelty: FakeToronto)
single_qubit_gates = ["rz", "sx", "x"]
multi_qubit_gates = ["cx"]

### Generation

In [ ]:
generate_qnn_circuits(max_qubits, max_depth, seed, single_qubit_gates, multi_qubit_gates)
generate_qaoa_circuits(max_qubits, max_depth, seed, single_qubit_gates, multi_qubit_gates)

### Verification

In [ ]:
# Helper function
def simulate_circuit(circuit: QuantumCircuit) -> None:
    """Simulate a quantum circuit noiselessly using statevector simulation."""

    simulator = AerSimulator()

    # Transpile the circuit for the simulator
    transpiled_circuit = transpile(circuit, simulator)

    # Run the simulation
    result = simulator.run(transpiled_circuit).result()

    # Get measurement results if circuit has measurements
    if circuit.num_clbits > 0:
        counts = result.get_counts()
        print("Measurement Results:", counts)

In [ ]:
circuit_name = "qaoa_3qubits_1layers_2degree"

with Path.open(current_dir / "qaoa_circuits" / f"{circuit_name}.qpy", "rb") as f:
    circ = qpy.load(f)[0]

simulate_circuit(circ)
circ.draw()